In [1]:
# Imports the packages Here
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
# Here we make a call to the dataset and retrieve the training, valid and test data.
cleaned_dataset_path = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/MNIST_ImageClassification/DataPreparation/dataset_cleaned.p'

with open(cleaned_dataset_path, 'rb') as f:
    fnl_dataset = pickle.load(f)
    training_dataset = (fnl_dataset['training_dataset'])
    training_labels = (fnl_dataset['training_labels'])
    test_dataset = (fnl_dataset['test_dataset'])
    test_labels = (fnl_dataset['test_labels'])
    crossvalid_dataset = (fnl_dataset['crossvalid_dataset'])
    crossvalid_labels = (fnl_dataset['crossvalid_labels'])
    
print('Training:', training_dataset.shape, training_labels.shape)
print('Cross Validation:', crossvalid_dataset.shape, crossvalid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (200000, 28, 28) (200000,)
Cross Validation: (9810, 28, 28) (9810,)
Testing: (7709, 28, 28) (7709,)


Notes:

1. For a convolution Neural Network we need a Tensor of shape (batchSize, imageSize, imageSize, numChannels). For a gray scale image the numChannels would be 1 for a RBG it would be 3. The below code will convert the input array into the tensor of said shape. The numChannels would even work without having it. But we have it because it will help us do the matrix multiplication with the specified depth (kernel) for the convolutional filter. Plus Tensor flow requires it this way.


2. For labels Tensor flow takes the labels as binary input where 1 indicated the activation of the class of the training instance and 0 for all other class. For example Alphabet A whose binary value is 0 will turn to an array with elements [1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0] and B becomes [0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0].

In [3]:
# Reshape the data, the input is in the form of tensor of shape [200000x28x28], Here we convert them into a single row
imageSize = 28
numChannels = 1 # For Grayscale image.
no_of_labels = 10

def reshape_data(dataset, labels, sample_size=None):
    if sample_size:
        dataset = dataset[:sample_size].reshape((-1,imageSize,imageSize,numChannels)) # To reshape the
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(no_of_labels) == labels[:,None]).astype(np.float32)
    else:
        dataset = dataset.reshape((-1,imageSize,imageSize,numChannels)) # To reshape the  
        # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
        labels = (np.arange(no_of_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

# We just reshape the image so that 1 image defines a row

# training_dataset[:].reshapeshape
trainData_, trainLabels_ = reshape_data(training_dataset, training_labels)
validData_, validLabels_ = reshape_data(crossvalid_dataset, crossvalid_labels)
testData_, testLabels_ = reshape_data(test_dataset, test_labels)
print('Training set', trainData_.shape, trainLabels_.shape)
print('Cross Validation set', validData_.shape, validLabels_.shape)
print('Test set', testData_.shape, testLabels_.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Cross Validation set (9810, 28, 28, 1) (9810, 10)
Test set (7709, 28, 28, 1) (7709, 10)


In [22]:
# Build the Convolution graph using Tensor Flow.

# Define Hyperparameter




class ConvNets():
    
    def __init__(self):
        self.numClasses = 10
        self.ch = 1               # numChannels -> For grayscale =1, for RGB =3
        self.h1 = 64              # numHiddenUnits for the fully connected layer
        
        self.ck1 = 5              # Size of kernel for the first convolution layer
        self.ck2 = 5              # Size of kernel for the second convolution layer
        self.cd1 = 16             # Number of kernels for the first convolution layer
        self.cd2 = 32             # Number of kernels for the second convolution layer
        self.cs1 = 1              # Strides for the first convolution layer  
        self.cs2 = 1              # Strides for the first convolution layer
        
        self.pk1 = 2              # Size of kernel for the first Pooling layer
        self.pk2 = 2              # Size of kernel for the second Pooling layer  
        self.ps1 = 2              # Strides for the first Pooling layer  
        self.ps2 = 2              # Strides for the first Pooling layer
        
        self.alpha = 0.05         # The learning rate
        
        
    def getWeightBiasCVP(self):
        weights = {
            'convLayer1_wgths' : tf.Variable(tf.truncated_normal([self.ck1,self.ck1,self.ch,self.cd1],stddev=0.1)), 
            # [kernelSize, kernelSize, numChannels, numKernels]
            'convLayer2_wghts' : tf.Variable(tf.truncated_normal([self.ck2,self.ck2,self.cd1,self.cd2],stddev=0.1)) 
            # [kernelSize, kernelSize, numChannels, numKernels]
        }         

        biases = {
            'convLayer1_bias' : tf.Variable(tf.zeros([self.cd1])),
            # [kernelSize]
            'convLayer2_bias' : tf.Variable(tf.constant(1.0, shape=[self.cd2]))
            # [kernelSize]
        }

        return weights, biases
    
    
    
    def getWeightBiasFC(self,flattenedPoolSize):
        # Here we need weigths for the fully connected network
        weightsFC ={}
        biasesFC = {}
        weightsFC['fcLayer_wgths'] = tf.Variable(tf.truncated_normal([flattenedPoolSize, self.h1]))
        # 16 is the depth or numkernels
        biasesFC['fcLayer_bias'] = tf.Variable(tf.constant(1.0, shape=[self.h1]))
        weightsFC['outputLayer_wghts'] = tf.Variable(tf.truncated_normal([self.h1, self.numClasses]))
        biasesFC['outputLayer_bias'] = tf.Variable(tf.constant(1.0, shape=[self.numClasses]))
        
        return weightsFC, biasesFC


    
    def convLayer(self, data, wgth, bias, stride=1, nlModel='RELU'):
        data = tf.nn.conv2d(data, wgth, [1,stride,stride,1], padding='SAME')
        # Add Bias
        if nlModel == 'RELU':
            return tf.nn.relu(data + bias)
        elif nlModel == 'LOGIT':
            hidden = tf.matmul(data, wgth) + bias
            return tf.sigmoid(hidden, name=None)


    def poolLayer(self, data, kernel, strides, poolType='MAX'):
        if poolType=='MAX':
            return tf.nn.max_pool(data, ksize=[1,kernel,kernel,1], strides=[1,strides,strides,1], padding='SAME')
        elif poolType == 'AVG':
            return tf.nn.avg_pool(data, ksize=[1,kernel,kernel,1], strides=[1,strides,strides,1], padding='SAME')

        
    def fcLayer(self, data, wght, bias, nModel):
        # convert the poolState2 into a flattened neuron structure
        data = tf.matmul(data, wght) + bias
        if nModel=='RELU':
            return tf.nn.relu(data)
        if nModel=='LOGIT':
            return tf.sigmoid(data, name=None)
        
    def outputLayer(self, data, wght, bias):
        data = tf.matmul(data, wght) + bias
        return data, tf.nn.softmax(data, name=None)
            
        

    def convNet(self, batchSize, imageSize, numChannels, numLabels):
        
        # Input Layer
        trainData = tf.placeholder(tf.float32, shape=(batchSize, imageSize, imageSize, self.ch))
        trainLabels = tf.placeholder(tf.float32, shape=(batchSize, numLabels))

        # Get weights and Biases for the input to Comvolution -> Pooling Layer
#         weights, biases = self.getWeightBiasCVP()
weightsFC[
        # 16 is the depth or numkernels
        biasesFC['fcLayer_bias'] = tf.Variable(tf.constant(1.0, shape=[self.h1]))
        weightsFC['outputLayer_wghts'] = tf.Variable(tf.truncated_normal([self.h1, self.numClasses]))
        biasesFC['outputLayer_bias'] = tf.Variable(tf.constant(1.0, shape=[self.numClasses]))
        
        weights = {
            'convLayer1_wgths' : tf.Variable(tf.truncated_normal([self.ck1,self.ck1,self.ch,self.cd1],stddev=0.1)), 
            'convLayer2_wghts' : tf.Variable(tf.truncated_normal([self.ck2,self.ck2,self.cd1,self.cd2],stddev=0.1)),
            'fcLayer_wgths' : tf.Variable(tf.truncated_normal([7*7*, self.h1]))
        }         

        biases = {
            'convLayer1_bias' : tf.Variable(tf.zeros([self.cd1])),
            'convLayer2_bias' : tf.Variable(tf.constant(1.0, shape=[self.cd2]))
        }
        
        

        # Convolution Layer1 and # Pooling Layer1
        convState1 = self.convLayer(trainData, weights['convLayer1_wgths'], biases['convLayer1_bias'], stride=self.cs1, nlModel='RELU')
        poolState1 = self.poolLayer(convState1, kernel=self.pk1, strides=self.ps1, poolType='MAX')

        # Convolution Layer2 and Pooling Layer 2
        convState2 = self.convLayer(poolState1, weights['convLayer2_wghts'], biases['convLayer2_bias'], stride=self.cs2, nlModel='RELU')
        poolState2 = self.poolLayer(convState2, kernel=self.pk2, strides=self.ps2, poolType='MAX')
        poolSize = poolState2.get_shape().as_list()[1]
        
        shapew = tf.shape(poolState2)
        
        # Get weigths and Biases for the Pooling Layer the Fully Connected Hidden Layer
        flattenedPoolSize = poolSize*poolSize*self.cd2 
        weightsFC, biasesFC = self.getWeightBiasFC(flattenedPoolSize)
        
        # Fully Connected Hidden Layer, We need to reshape the pool layer into a flattened neuron layer
        fcLayerdata = tf.reshape(poolState2, [-1, flattenedPoolSize])   # -1 states no change to the shape at that dimension
        fcState = self.fcLayer(fcLayerdata, weightsFC['fcLayer_wgths'], biasesFC['fcLayer_bias'], nModel='RELU')
        fc_to_output, outputState = self.outputLayer(fcState, weightsFC['outputLayer_wghts'], biasesFC['outputLayer_bias'])
        
        # Calculate the Loss
        loss_CE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(fc_to_output, trainLabels))
        
        # Optimize The loss Function
        optimizer = tf.train.GradientDescentOptimizer(self.alpha).minimize(loss_CE)
        
        # Training Prediction
        pred = tf.equal(tf.argmax(outputState, 1), tf.argmax(trainLabels, 1))
        accuracy = tf.reduce_mean(tf.cast(pred, tf.float32))
        
        
        return dict(
            trainData=trainData,
            trainLabels=trainLabels,
            convState1=convState1,
            poolState1=poolState1,
            convState2=convState2,
            poolState2=poolState2,
            shapew = shapew,
            weightsFC=weightsFC['fcLayer_wgths'],
            fcLayerdata=fcLayerdata,
            fcState=fcState,
            fc_to_output=fc_to_output,
            loss_CE=loss_CE,
            optimizer=optimizer,
            pred=pred,
            accuracy=accuracy
        )
        # Fully Connected Layer
        # The poolState will result in a Tensor of shape
    

SyntaxError: invalid syntax (<ipython-input-22-27d8319428ba>, line 104)

In [21]:

def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()


def convLayer(x, w, b, s=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, w, strides=[1, s, s, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxPoolLayer(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


def convNet(trainData, trainLabels, weights, biases, batchSize, imageSize, numLabels):
    trainData = tf.reshape(trainData, shape=[batchSize, imageSize, imageSize, numChannel])
    
    # Convolution Layer1 and # Pooling Layer1
    convState1 = convLayer(trainData, weights['conv1_wgths'], biases['conv1_bias'])
    convState1 = maxPoolLayer(convState1, k=2)

    # Convolution Layer2 and Pooling Layer 2
    convState2 = convLayer(convState1, weights['conv2_wghts'], biases['conv2_bias'])
    convState2 = maxPoolLayer(convState2, k=2)

    # Fully Connected Hidden Layer, We need to reshape the pool layer into a flattened neuron layer
    fcLayerData = tf.reshape(convState2, [-1, weights['fc_wgths'].get_shape().as_list()[0]])
    fcState = tf.add(tf.matmul(fcLayerData, weights['fc_wgths']), biases['fc_bias'])
    fcState = tf.nn.relu(fcState)
    
    # Prediction
    outputState = tf.add(tf.matmul(fcState, weights['output_wghts']), biases['output_bias'])

    return outputState


# Reset The Graph
reset_graph()


# Define Hyperparameters
          # The learning rate


# Define the input
trainData = tf.placeholder(tf.float32, [None, 784])
trainLabels = tf.placeholder(tf.float32, shape=(batchSize, numLabels))
    
# Define Weigths
weights = {
    'conv1_wgths' : tf.Variable(tf.truncated_normal([conv1Kernel,conv1Kernel,numChannel,conv1Depth])), 
    'conv2_wghts' : tf.Variable(tf.truncated_normal([conv2Kernel,conv2Kernel,conv1Depth,conv2Depth])),
    'fc_wgths' : tf.Variable(tf.truncated_normal([7*7*conv2Depth, numHidden])),
    'output_wghts' : tf.Variable(tf.truncated_normal([numHidden, numLabels]))
}         

biases = {
    'conv1_bias' : tf.Variable(tf.random_normal([conv1Depth])),
    'conv2_bias' : tf.Variable(tf.random_normal([conv2Depth])),
    'fc_bias' : tf.Variable(tf.random_normal([numHidden])),
    'output_bias': tf.Variable(tf.random_normal([numLabels]))
}

# Get The model estimate
estimate = convNet(trainData, trainLabels, weights, biases, batchSize=batchSize, imageSize=28, numLabels=10)

# Calculate the Loss and Optimize
loss_CE = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=estimate, labels=trainLabels))
optimizer = tf.train.AdamOptimizer(learning_rate=alpha).minimize(loss_CE)

# Training Prediction
pred = tf.equal(tf.argmax(estimate, 1), tf.argmax(trainLabels, 1))
accuracy = tf.reduce_mean(tf.cast(pred, tf.float32))



#     return dict(
#         trainData=trainData,
#         trainLabels=trainLabels,
#         convState1=convState1,
#         convState2=convState2,
#         fcLayerData=fcLayerData,
#         fcState=fcState,
#         loss_CE=loss_CE,
#         optimizer=optimizer,
#         pred=pred,
#         accuracy=accuracy
#     )

In [ ]:
training_iters = 200000
display_step = 10

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    step = 1
    # Keep training until reach max iterations
    while step * batchSize < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batchSize)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={trainData: batch_x, trainLabels: batch_y})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_CE, accuracy], feed_dict={trainData: batch_x,
                                                              trainLabels: batch_y})
            print ("Iter " + str(step*batchSize) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
#     print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
#                                       y: mnist.test.labels[:256],
#                                       keep_prob: 1.})


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter 1280, Minibatch Loss= 40.432938, Training Accuracy= 0.13281
Iter 2560, Minibatch Loss= 18.971008, Training Accuracy= 0.07031
Iter 3840, Minibatch Loss= 9.288128, Training Accuracy= 0.09375
Iter 5120, Minibatch Loss= 5.570509, Training Accuracy= 0.09375


In [5]:
batchSize = 128
numTraining = trainData_.shape[0]
displayStep = 10

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# def accuracy(predictions, labels, labels_one_hot = None):
#     # The input labels are a One-Hot Vector
#     if labels_one_hot:
# #         print (predictions)
# #         print (predictions.shape[0])
# #         print (np.argmax(labels, 1))
# #         print (np.argmax(predictions, 1))
#         return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])
#     else:
#         return (100.0 * np.sum(np.argmax(predictions, 1) == np.reshape(labels, [-1])) / predictions.shape[0])

    
def train_network():
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        epoch = 1
        while epoch*batchSize < numTraining:
            
#             print ((epoch-1)*batchSize, epoch*batchSize)
            batchData = trainData_[(epoch-1)*batchSize : epoch*batchSize]
            batchLabels = trainLabels_[(epoch-1)*batchSize : epoch*batchSize]

            sess.run(optimizer, feed_dict={trainData: batchData, trainLabels: batchLabels})

            
            if epoch % displayStep == 0:
                loss, acc = sess.run([loss_CE, accuracy], feed_dict={trainData: batchData, trainLabels: batchLabels})
                print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc)
                
            epoch += 1
#             break
            


# print (graphDict)
train_network()

Iter 1280, Minibatch Loss= 21.263004, Training Accuracy= 0.07812
Iter 2560, Minibatch Loss= 11.673630, Training Accuracy= 0.09375
Iter 3840, Minibatch Loss= 5.153423, Training Accuracy= 0.10156
Iter 5120, Minibatch Loss= 3.666912, Training Accuracy= 0.06250
Iter 6400, Minibatch Loss= 3.051682, Training Accuracy= 0.12500


KeyboardInterrupt: 

In [91]:
array_np = np.random.rand(5,3,3)
array_np = array_np.reshape((-1, 5,3,1)).astype(np.float32)
print(array_np.shape)
array_np

(3, 5, 3, 1)


array([[[[ 0.52622551],
         [ 0.68646508],
         [ 0.65008277]],

        [[ 0.06481812],
         [ 0.54809189],
         [ 0.76726586]],

        [[ 0.83489174],
         [ 0.75896871],
         [ 0.25165305]],

        [[ 0.13270801],
         [ 0.58833188],
         [ 0.14629339]],

        [[ 0.7359888 ],
         [ 0.08913432],
         [ 0.66141826]]],


       [[[ 0.05585782],
         [ 0.69977093],
         [ 0.52322626]],

        [[ 0.97286582],
         [ 0.22512116],
         [ 0.48032147]],

        [[ 0.15738663],
         [ 0.01338428],
         [ 0.57128853]],

        [[ 0.30658212],
         [ 0.24820569],
         [ 0.77591407]],

        [[ 0.39811641],
         [ 0.86327881],
         [ 0.64881307]]],


       [[[ 0.27618587],
         [ 0.10348999],
         [ 0.01123772]],

        [[ 0.17071991],
         [ 0.50098807],
         [ 0.28569326]],

        [[ 0.65558779],
         [ 0.88648224],
         [ 0.42478511]],

        [[ 0.58333689],
         [

In [32]:
#             print ('')
#             print ('training training training, training training training, training training training training training')
#                 for no in np.arange(num_batches):
#                     with open(self.train_batch_dir+'batch'+str(no)+'.pickle', 'rb') as f:
#                         dataset = pickle.load(f)
                        
#                         batch_train_dataset = dataset['batch_train_dataset']
#                         batch_train_labels = dataset['batch_train_labels']
#                         batch_train_lenarr = dataset['batch_train_lenarr']
                        
#                         feed_dict= {graph_dict['x']: batch_train_dataset, 
#                                     graph_dict['y']: batch_train_labels,
#                                     graph_dict['x_lenarr']: batch_train_lenarr}
            
#                         if new_state_ is not None:
#                             feed_dict[graph_dict['init_state']] = new_state_

#                         new_state_, loss_, opt, tp = sess.run([graph_dict['new_state'],
#                                                             graph_dict['loss'],
#                                                             graph_dict['optimizer'],
#                                                             graph_dict['prediction']], 
#                                                             feed_dict=feed_dict)
                        
#                         training_loss += loss_
#                         acc = self.accuracy(tp, batch_train_labels)
                    
                        
#                         print ('accuracy of the training batch %d is: '%no, acc)
                        
                        
#                         if (no%valid_after_batches==0 and no!=0):
#                             print ('Average Loss till the training batch %d is: '%no, training_loss/no)
#                             print ('')
#                             print ('crossvalid crossvalid crossvalid, crossvalid crossvalid crossvalid, crossvalid crossvalid crossvalid')
#                             self.cross_valid(sess,graph_dict)
#                             print ('')
#                             print ('training training training, training training training, training training training training training')
                            
#                 print ('All %d Batches Done.................................'%num_batches)
#                 print ('')
#                 print ('')
                
#                 if verbose:
#                     print("Average training loss for Epoch", epoch, ":", training_loss/num_batches)
            
#             if isinstance(path_save_model, str):
#                 graph_dict['saver'].save(sess, path_save_model)
    

# graph = tf.Graph()
# with graph.as_default():
#     # Define input data, valid data and test data
#     tf_trainDataset = tf.placeholder(tf.float32, shape=(batchSize, image_size, image_size))
#     tf_trainLabels = tf.placeholder(tf.float32, shape=(batchSize, numLabels, image_size))

#     # Define weights first normal weights are taken.
#     # The weights are shared accross all the neurons for one Kernel and we have one bais for each kernel.
    
#     convlayer1_bias = tf.Variable(tf.Zeros(kernelSize))
#     convlayer2_bias = tf.Variable(tf.Zeros(kernelSize))
# #     layer3_wghts = 

7

In [181]:
28 // 4 * 28 // 4 * 32

1568

In [15]:
def convLayer(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxPoolLayer(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    # Convolution Layer
    conv1 = convLayer(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxPoolLayer(conv1, k=2)

    # Convolution Layer
    conv2 = convLayer(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxPoolLayer(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
#     fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [16]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, 10]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([10]))
}

x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=(None, 10))

# Construct model
pred = conv_net(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.05).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

In [ ]:
numTraining = 200000
training_iters = numTraining
displayStep = 10
display_step = 10
batchSize = 128
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# with tf.Session() as sess:
#     sess.run(tf.initialize_all_variables())
#     epoch = 1
#     while epoch*batchSize < numTraining:

# #             print ((epoch-1)*batchSize, epoch*batchSize)
#         batchData = trainData_[(epoch-1)*batchSize : epoch*batchSize]
#         batchLabels = trainLabels_[(epoch-1)*batchSize : epoch*batchSize]

#         sess.run(optimizer, feed_dict={x: batchData, y: batchLabels})


#         if epoch % displayStep == 0:
#             loss, acc = sess.run([cost, accuracy], feed_dict={x: batchData, y: batchLabels})
#             print ("Iter " + str(epoch*batchSize) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
            
#         epoch += 1
#         break

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    step = 1
    # Keep training until reach max iterations
    while step * batchSize < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batchSize)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y})
            print ("Iter " + str(step*batchSize) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter 1280, Minibatch Loss= 199.656830, Training Accuracy= 0.13281
Iter 2560, Minibatch Loss= 14.407655, Training Accuracy= 0.09375
Iter 3840, Minibatch Loss= 8.168202, Training Accuracy= 0.09375


In [61]:
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [62]:

# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
# dropout = 0.75 # Dropout, probability to keep units

def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()


    
# tf Graph input

keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [65]:
def convLayer(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def poolLayer(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(weights, biases):
    # Reshape input picture
#     x = tf.reshape(x, shape=[-1, 28, 28, 1])
    x = tf.placeholder(tf.float32, [128, 28, 28, 1])
    y = tf.placeholder(tf.float32, [128, n_classes])


    # Convolution Layer
    conv1 = convLayer(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = poolLayer(conv1, k=2)

    # Convolution Layer
    conv2 = convLayer(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = poolLayer(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
#     fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    
#     Construct model
# pred = conv_net(x, weights, biases, keep_prob)

    
    
    return out

In [66]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

x = tf.placeholder(tf.float32, [128, 28, 28, 1])
y = tf.placeholder(tf.float32, [128, n_classes])


pred = conv_net(weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))



# Initializing the variables
init = tf.initialize_all_variables()

In [67]:
batchSize = 128
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
#         batchData, batchLabels = mnist.train.next_batch(batch_size)
        batchData = trainData_[(step-1)*batchSize : step*batchSize]
        batchLabels = trainLabels_[(step-1)*batchSize : step*batchSize]
        # Run optimization op (backprop)
        opt = sess.run(optimizer, feed_dict={x: batchData, y: batchLabels})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batchData,
                                                              y: batchLabels})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")

#     Calculate accuracy for 256 mnist test images
#     print ("Testing Accuracy:", \
#         sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
#                                       y: mnist.test.labels[:256],
#                                       keep_prob: 1.}))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_36' with dtype float and shape [128,28,28,1]
	 [[Node: Placeholder_36 = Placeholder[dtype=DT_FLOAT, shape=[128,28,28,1], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op 'Placeholder_36', defined at:
  File "/Users/sam/App-Setup/CondaENV/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/sam/App-Setup/CondaENV/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/sam/App-Setup/CondaENV/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/sam/App-Setup/CondaENV/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/sam/App-Setup/CondaENV/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/Users/sam/App-Setup/CondaEN